<a href="https://colab.research.google.com/github/elangbijak4/Universal-Language-and-Protocol-Construction/blob/main/Demo_1_Bahasa_Universal_Alien_Bilangan_Prima_dan_Fibonacci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# CELL 1
# PFUL demo: Utilities (primes, factorization, fibonacci)
# Paste this as the first cell and run.

def sieve_primes(n):
    """Return list of primes up to n (inclusive)."""
    if n < 2:
        return []
    sieve = [True] * (n+1)
    sieve[0:2] = [False, False]
    import math
    for p in range(2, int(math.isqrt(n)) + 1):
        if sieve[p]:
            step = p*p
            sieve[step:n+1:p] = [False] * (((n - step)//p) + 1)
    return [i for i, isprime in enumerate(sieve) if isprime]

def factorize(n, primes_list=None):
    """Simple trial division factorization. Returns dict {prime: exponent}."""
    if n < 2:
        return {}
    import math
    if primes_list is None:
        primes_list = sieve_primes(int(max(100, math.isqrt(n) + 10)))
    rem = n
    factors = {}
    for p in primes_list:
        if p*p > rem:
            break
        if rem % p == 0:
            cnt = 0
            while rem % p == 0:
                rem //= p
                cnt += 1
            factors[p] = cnt
    if rem > 1:
        factors[rem] = factors.get(rem, 0) + 1
    return factors

def generate_fibonacci(k):
    """Return list of first k Fibonacci numbers (starting F0=0, F1=1)."""
    if k <= 0:
        return []
    if k == 1:
        return [0]
    F = [0, 1]
    for _ in range(2, k):
        F.append(F[-1] + F[-2])
    return F

# Prepare a primes cache for demo
_first_primes = sieve_primes(5000)
def prime_list(limit):
    global _first_primes
    if _first_primes[-1] < limit:
        _first_primes = sieve_primes(limit)
    return _first_primes

print("Utilities loaded. Example: first 10 primes:", prime_list(30)[:10])
print("First 10 Fibonacci numbers:", generate_fibonacci(10))

Utilities loaded. Example: first 10 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
First 10 Fibonacci numbers: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


In [4]:
# CELL 2
# PTL: symbol mapping, naive product encoding, and Gödel-style ordered encoding

# map a small alphabet to first primes
first_primes = prime_list(200)
symbols = ['A','B','C','D','E','F','G','H','I','J']  # demo alphabet
alphabet = {s:first_primes[i] for i,s in enumerate(symbols)}
rev_alphabet = {v:k for k,v in alphabet.items()}

print("Alphabet mapping (symbol -> prime):")
for k in alphabet:
    print(k, "->", alphabet[k])
print()

def encode_word_product(word, mapping=alphabet):
    """Naive product encoding (order not preserved)."""
    prod = 1
    for ch in word:
        if ch not in mapping:
            raise KeyError(f"Symbol '{ch}' not in mapping")
        prod *= mapping[ch]
    return prod

def decode_word_product(n, revmap=rev_alphabet):
    fac = factorize(n, prime_list(1000))
    # expand to multiset (order lost)
    seq = []
    for p,e in sorted(fac.items()):
        seq += [revmap.get(p, f"?({p})")] * e
    return seq, fac

# Gödel-style position-preserving encoding:
def godel_encode(seq, base_primes=first_primes, symbol_index_map=None):
    """Encode sequence while preserving order:
       use base_primes[pos] ** index_of_symbol
       symbol_index_map: mapping from symbol->small positive integer (1-based)
    """
    if symbol_index_map is None:
        # create an index map from alphabet order
        symbol_index_map = {s:i+1 for i,s in enumerate(alphabet.keys())}
    prod = 1
    for pos, ch in enumerate(seq):
        p = base_primes[pos]
        idx = symbol_index_map[ch]
        prod *= p ** idx
    return prod

def godel_decode(n, base_primes=first_primes, reverse_index_map=None, max_len=20):
    fac = factorize(n, prime_list(2000))
    if reverse_index_map is None:
        reverse_index_map = {i+1:s for i,s in enumerate(alphabet.keys())}
    decoded = []
    for pos, p in enumerate(base_primes[:max_len]):
        if p in fac:
            exp = fac[p]
            decoded.append(reverse_index_map.get(exp, f"?idx{exp}"))
        else:
            break
    return decoded, fac

# Demo
word = "ACAD"
prod_enc = encode_word_product(word)
print("Naive product encoding for", word, "->", prod_enc)
decoded_multiset, f = decode_word_product(prod_enc)
print("Decoded multiset (order not preserved):", decoded_multiset, "factors:", f)

genc = godel_encode(list(word))
decoded_ordered, f2 = godel_decode(genc)
print("Gödel encoding preserves order ->", genc)
print("Decoded ordered:", decoded_ordered, "factors:", f2)

Alphabet mapping (symbol -> prime):
A -> 2
B -> 3
C -> 5
D -> 7
E -> 11
F -> 13
G -> 17
H -> 19
I -> 23
J -> 29

Naive product encoding for ACAD -> 140
Decoded multiset (order not preserved): ['A', 'A', 'C', 'D'] factors: {2: 2, 5: 1, 7: 1}
Gödel encoding preserves order -> 648270
Decoded ordered: ['A', 'C', 'A', 'D'] factors: {2: 1, 3: 3, 5: 1, 7: 4}


In [5]:
# CELL 3
# PTL: Simple Turing-like toggle example (single cell toggling 0<->1)
# Will encode rules as prime products and show applying rules by arithmetic transforms.

# Symbol and state primes (small demo values)
symmap = {'0':2, '1':3}
statemap = {'q0':5, 'q1':7}
dir_map = {'L':11, 'R':13}

# Encode two rules: (q0,0)->(q0,1,R) and (q0,1)->(q0,0,R)
rule1 = statemap['q0'] * symmap['0'] * statemap['q0'] * symmap['1'] * dir_map['R']
rule2 = statemap['q0'] * symmap['1'] * statemap['q0'] * symmap['0'] * dir_map['R']
print("Encoded rule1:", rule1)
print("Encoded rule2:", rule2)

def encode_conf(state, tape):
    """Encode configuration:
       state prime * product_{pos}(sym_prime ** (pos+2))
       tape: dict position(int)->symbol('0'/'1')
    """
    conf = statemap[state]
    for pos, sym in tape.items():
        conf *= symmap[sym] ** (pos + 2)
    return conf

def decode_conf(conf):
    fac = factorize(conf, prime_list(500))
    # find state
    state = None
    for s,p in statemap.items():
        if p in fac:
            state = s
            break
    # extract tape symbols by primes 2 and 3 and exponent -> position
    tape = {}
    for p,e in fac.items():
        if p in (2,3):
            pos = e - 2
            tape[pos] = '0' if p==2 else '1'
    return state, tape, fac

def apply_rule(conf, rule):
    """Very simplified rule application:
       - locate p_a (2 or 3) and p_b (2 or 3) in rule
       - replace exponent occurrences in conf accordingly (single-cell demo)
       - update state prime from p_q to p_qp if present
    """
    fac_conf = factorize(conf, prime_list(500))
    fac_rule = factorize(rule, prime_list(500))
    p_a = p_b = p_q = p_qp = None
    # identify pattern in rule
    for p in fac_rule:
        if p in (2,3):
            if p_a is None:
                p_a = p
            else:
                p_b = p
        elif p in statemap.values():
            if p_q is None:
                p_q = p
            else:
                p_qp = p
    # if p_a not present in config, rule not applicable
    exp = fac_conf.get(p_a, 0)
    if exp == 0:
        return conf  # no change
    # perform substitution: remove p_a^exp and add p_b^exp
    new_conf = conf // (p_a ** exp) * (p_b ** exp)
    # replace state prime if present
    if p_q in fac_conf and p_qp is not None:
        new_conf = new_conf // p_q * p_qp
    return new_conf

# simulate
tape0 = {0: '0'}
conf = encode_conf('q0', tape0)
print("Initial conf integer:", conf)
print("Decoded:", decode_conf(conf))

conf_after = apply_rule(conf, rule1)
print("After applying rule1 ->", conf_after)
print("Decoded:", decode_conf(conf_after))

conf_after2 = apply_rule(conf_after, rule2)
print("After applying rule2 ->", conf_after2)
print("Decoded:", decode_conf(conf_after2))

Encoded rule1: 1950
Encoded rule2: 1950
Initial conf integer: 20
Decoded: ('q0', {0: '0'}, {2: 2, 5: 1})
After applying rule1 -> 45
Decoded: ('q0', {0: '1'}, {3: 2, 5: 1})
After applying rule2 -> 45
Decoded: ('q0', {0: '1'}, {3: 2, 5: 1})


In [6]:
# CELL 4
# FBL: Represent organisms as Fibonacci compositions and apply morphisms (growth, reproduction)

F = generate_fibonacci(50)  # enough for demo
print("First 12 Fibonacci:", F[:12])

# Example organism represented by Fibonacci values
organism = [1, 2, 5, 2]  # values, not indices
print("Organism (F-values):", organism)

def phi_growth_by_value(seq, fib_seq=F):
    """Growth morphism: each Fibonacci value F_n -> (F_{n+1}, F_n)."""
    out = []
    for v in seq:
        if v in fib_seq:
            i = fib_seq.index(v)
            out.extend([fib_seq[i+1], fib_seq[i]])
        else:
            out.append(v)  # unknown value, pass through
    return out

print("After growth morphism:", phi_growth_by_value(organism))

# Another morphism: fusion of two organisms pairwise by index-sum
def fusion_pairwise(o1, o2, fib_seq=F):
    """Given two sequences of equal length, produce F_{i+j} elements (by index)."""
    out = []
    for a,b in zip(o1, o2):
        if a in fib_seq and b in fib_seq:
            ia = fib_seq.index(a); ib = fib_seq.index(b)
            out.append(fib_seq[ia+ib])
        else:
            out.append(None)
    return out

print("Fusion example:", fusion_pairwise([1,2,3],[2,3,5]))

First 12 Fibonacci: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]
Organism (F-values): [1, 2, 5, 2]
After growth morphism: [1, 1, 3, 2, 8, 5, 3, 2]
Fusion example: [3, 13, 34]


In [7]:
# CELL 5
# PFUL: Embed (FBL -> PTL) and Lift (PTL -> FBL) operators

# Embed: map Fibonacci values into primes by using index->prime mapping
def embed_fib_sequence(seq, fib_seq=None, prime_bases=None):
    if fib_seq is None:
        fib_seq = generate_fibonacci(200)
    if prime_bases is None:
        prime_bases = prime_list(2000)
    prod = 1
    for v in seq:
        if v in fib_seq:
            idx = fib_seq.index(v)  # index of that Fibonacci value
            p = prime_bases[idx]    # map index -> prime at same index
            prod *= p
        else:
            # if value not found, skip or multiply by 1
            prod *= 1
    return prod

# Lift: interpret prime factorization exponents (or positions) as Fibonacci indices
def lift_number_to_fib(n, prime_bases=None, fib_seq=None):
    if prime_bases is None:
        prime_bases = prime_list(2000)
    if fib_seq is None:
        fib_seq = generate_fibonacci(200)
    fac = factorize(n, prime_bases)
    seq = []
    for p, e in sorted(fac.items()):
        # map prime p to its index in prime_bases if present
        if p in prime_bases:
            idx = prime_bases.index(p)
            # interpret exponent as Fibonacci index if possible
            if e < len(fib_seq):
                seq.append(fib_seq[e])
            else:
                seq.append(None)
        else:
            seq.append(None)
    return seq, fac

# Demo embedding & lifting
fib_seq = [1,2,3,5]
embed_val = embed_fib_sequence(fib_seq)
print("Embed", fib_seq, "->", embed_val)
lifted_seq, fac = lift_number_to_fib(embed_val)
print("Lifted (approx):", lifted_seq)
print("Prime factors:", fac)

Embed [1, 2, 3, 5] -> 3003
Lifted (approx): [1, 1, 1, 1]
Prime factors: {3: 1, 7: 1, 11: 1, 13: 1}


In [8]:
# CELL 6
# CELL to produce a small PFUL message and save as JSON (file will be in /content/ in Colab)

import json, os
message = {
    'primes_signature': [2,3,5,7,11],
    'fibonacci_signature': [1,1,2,3,5,8],
    'ptl_sentence_example': 2310,
    'fbl_rule_example': {'input': [5,8,13], 'output': [13,21]},
    'hybrid_example': {'tech': 2310, 'bio': [1,2,3]}
}
outpath = '/content/pful_message.json'
with open(outpath, 'w') as f:
    json.dump(message, f, indent=2)
print("Saved PFUL demo message to:", outpath)
print("Contents:")
print(json.dumps(message, indent=2))

Saved PFUL demo message to: /content/pful_message.json
Contents:
{
  "primes_signature": [
    2,
    3,
    5,
    7,
    11
  ],
  "fibonacci_signature": [
    1,
    1,
    2,
    3,
    5,
    8
  ],
  "ptl_sentence_example": 2310,
  "fbl_rule_example": {
    "input": [
      5,
      8,
      13
    ],
    "output": [
      13,
      21
    ]
  },
  "hybrid_example": {
    "tech": 2310,
    "bio": [
      1,
      2,
      3
    ]
  }
}
